In [ ]:
from qepy.driver import Driver
from qepy.io import QEInput
import numpy as np

In [ ]:
from ase.build import bulk
from ase.io import read,write
from ase import Atoms
import py3Dmol
import matplotlib.pyplot as plt

# Challenge solution

In [ ]:
atoms = bulk('Al', 'bcc', a=4.05)
write('Al.xyz',atoms)
v = py3Dmol.view(data=open('./Al.xyz','r').read(), style='sphere')
v.addUnitCell()
v.zoomTo()
v.show()

In [ ]:
# atoms.get_cell()

In [ ]:
qe_options = {
    '&control': {
        'calculation': "'scf'",
        'pseudo_dir': "'./'",
        'outdir': "tmp",
    },
    '&system': {
        'ibrav' : 0,
        'degauss': 0.005,
        'ecutwfc': 40,
        'nat': 1,
        'ntyp': 1,
        'occupations': "'smearing'",
    },
    'atomic_positions crystal': ['Al    0.0  0.0  0.0'],
    'atomic_species': ['Al  26.98 Al.pbe-nl-kjpaw_psl.1.0.0.UPF'],
    'k_points automatic': ['10 10 10 1 1 1'],
    'cell_parameters angstrom':[
        '-2.025     2.025  2.025',
        '2.025  -2.025     2.025',
        '2.025  2.025  -2.025   '],
}

In [ ]:
ecutwfcs = np.arange(15, 40, 5)
energies_cut = []
for ecutwfc in ecutwfcs:
    qe_options['&system']['ecutwfc'] = ecutwfc
    #
    driver = Driver(qe_options=qe_options, logfile=True)
    ene = driver.scf()
    energies_cut.append(ene)
    print(ecutwfc, ene)
    driver.stop()

In [ ]:
kps = np.arange(2, 12, 2)
energies_k = []
for kp in kps:
    qe_options['k_points automatic'] = [f'{kp} {kp} {kp} 1 1 1']
    #
    driver = Driver(qe_options=qe_options, logfile=True)
    ene = driver.scf()
    energies_k.append(ene)
    print(kp, ene)
    driver.stop()

In [ ]:
%%time
degs= [1E-1, 5E-2, 1E-2, 5E-3, 1E-3]
energies_d = []
for degauss in degs:
    qe_options['&system']['degauss'] = degauss
    #
    driver = Driver(qe_options=qe_options, logfile=True)
    ene = driver.electrons()
    print('degauss',degauss, ene, driver.get_scf_steps())
    energies_d.append(ene)
    driver.stop()

In [ ]:
energies1=np.asarray(energies_d)
energies2=np.asarray(energies_k)
energies3=np.asarray(energies_cut)

In [ ]:
fig, ax = plt.subplots()

ax.plot(degs, np.abs(energies1-energies1[-1]), 'r-.', label='degauss')

ax.set_xscale('log')
ax.invert_xaxis()
plt.xlabel('Log Gaussian Spreading (Ry)')
plt.ylabel('Total Energy (Ry)')
ax.legend()

In [ ]:
plt.plot(kps, energies2-energies2[-1])
plt.xlabel('k-points ')
plt.ylabel('Total Energy (Ry)')

In [ ]:
plt.plot(ecutwfcs, energies3-energies3[-1])
plt.xlabel('Kin. Energy Cutoff (Ry)')
plt.ylabel('Total Energy (Ry)')